# Init

In [1]:
import os

path = os.getcwd()
# find the string 'project' in the path, return index
index_project = path.find("project")
# slice the path from the index of 'project' to the end
project_path = path[: index_project + 7]
# set the working directory
os.chdir(project_path + "/src")
print(f"Project path set to: {os.getcwd()}")


Project path set to: c:\Github\new-peak-project\src


In [2]:
from dotenv import dotenv_values
config = dotenv_values(".env")
print(config["DATA_PATH"])

I:\My Drive\DAWSON PHD PROJECT\Biomarker Data Repository\data\new-peak-project\experiments


In [3]:
new_path = config["NEW_DATA_PATH"]
print(f'New data path set to: {new_path}')

New data path set to: I:\My Drive\DAWSON PHD PROJECT\Biomarker Data Repository\data\data-and-mechanism-project


# Config

In [4]:
from models.utils.config_manager import initialise_config

initialise_config(folder_name="extended-simulation-1", verbose=1)


Folder I:\My Drive\DAWSON PHD PROJECT\Biomarker Data Repository\data\data-and-mechanism-project\extended-simulation-1 already exists. No changes made.


In [5]:
# or load existing config
from models.utils.config_manager import load_configs, print_config

loaded_config = load_configs(folder_name="extended-simulation-1", config_suffix="CuratedModel1")
print_config(loaded_config)

notebook: 
  version: CuratedModel1
  name: extended-simulation-1
  compatible_notebooks: ['extended-simulation-1']
  desc: CuratedModel1 - Sungyoung's example model designed by hand, captured algorithmically
Testing model responsiveness through parameter sampling over many iterations 

exp: 
  spec: 
    n_layers: 2
    n_cascades: 3
    n_regs: 0
    basal_activation: True
    custom_regulations: [['R1', 'R2', 'up'], ['R3', 'I1_2', 'up'], ['I1_1', 'I2_2', 'up'], ['I1_2', 'I2_1', 'down'], ['I1_2', 'I2_3', 'down'], ['I1_3', 'I2_2', 'up'], ['I2_1', 'R1', 'down'], ['I2_3', 'R3', 'up']]
    gen_seed: 42
    drug: 
      name: D
      start: 500
      dose: 500
      regulations: [['R1', 'down']]
      target_all: False
  parameter_generation: 
    ic_range: [200, 1000]
    param_range: [0.1, 10]
    param_mul_range: [0.99, 1.01]
  simulation: 
    start: 0
    stop: 1000
    step: 100
  parameter_sampling: 
    sampling_seed: 42
    num_models: 1000
    num_datapoints: 1000


# Run

### Setup

In [6]:
from models.Specs.ModelSpec4 import ModelSpec4

notebook_config = loaded_config["notebook"]
config_name = notebook_config['version']
exp_config = loaded_config["exp"]
spec_config = exp_config['spec']
n_layers = spec_config['n_layers']
new_spec = ModelSpec4(num_intermediate_layers=n_layers)


In [7]:
from models.Specs.Drug import Drug

drug_config = spec_config['drug']
drug_name = drug_config['name']
drug_start = drug_config['start']
drug_dose = drug_config['dose']
drug_regulations = drug_config['regulations']


n_cascades = spec_config["n_cascades"]
n_regs = spec_config["n_regs"]
seed = spec_config["gen_seed"]

for regulation in spec_config["custom_regulations"]:
    new_spec.add_regulation(*regulation)

new_drug = Drug(name=drug_name, start_time=drug_start, default_value=drug_dose)

# check if target_all exists in drug_config, if not set to False
drug_target_all = drug_config.get('target_all', False)

if drug_target_all:
    # If the drug targets all receptors, we don't need to add specific regulations
    for n in range(n_cascades):
        target = f'R{n+1}' # assuming receptors are named R1, R2, ..., Rn
        new_drug.add_regulation(target, 'down') # assuming the type is 'down' for all
else: 
    for regs in drug_regulations:
        target, type = regs[0], regs[1]
        new_drug.add_regulation(target, type)

new_spec.generate_specifications(n_cascades, n_regs, seed)
new_spec.add_drug(new_drug)

In [8]:
import numpy as np

param_sampling_config = exp_config["parameter_sampling"]
base_sampling_seed = param_sampling_config["sampling_seed"]
num_models = param_sampling_config["num_models"]
# based on base seed create 1000 different seeds
seeds = np.random.default_rng(base_sampling_seed).integers(0, 1000000, size=num_models)

param_gen_config = exp_config['parameter_generation']
basal_activation = spec_config["basal_activation"]
specie_range = param_gen_config['ic_range']
param_range = param_gen_config['param_range']
param_mul_range = param_gen_config['param_mul_range']

builder_models = []
for seed in seeds:
    builder = new_spec.generate_network(
        config_name,
        specie_range,
        param_range,
        param_mul_range,
        seed,
        receptor_basal_activation=basal_activation,
    )
    builder_models.append(builder)
print(f"Generated {len(builder_models)} models.")

Generated 1000 models.


### Simulations

In [ ]:
from models.Solver.RoadrunnerSolver import RoadrunnerSolver
from models.utils.config_manager import save_data
from tqdm import tqdm
import pandas as pd


sim_config = exp_config["simulation"]
sim_start = sim_config["start"]
sim_stop = sim_config["stop"]
sim_step = sim_config["step"]

# --- Simulations ---
solver_models = []
for i, builder in enumerate(tqdm(builder_models, desc="Creating solvers")):
    solver = RoadrunnerSolver()
    solver.compile(builder.get_sbml_model())
    solver_models.append(solver)


sim_results = []
for i, solver in enumerate(tqdm(solver_models, desc="Simulating models")):
    res = solver.simulate(sim_start, sim_stop, sim_step)
    sim_results.append(res)


# Concatenate all simulation results into a single DataFrame with 'seed' as its column using seeds list
sim_results_df = pd.concat(
    [pd.DataFrame(res).assign(seed=seed) for res, seed in zip(sim_results, seeds)],
    ignore_index=True,
)




Simulating models: 100%|██████████| 1000/1000 [00:02<00:00, 400.64it/s]


Data saved at I:\My Drive\DAWSON PHD PROJECT\Biomarker Data Repository\data\data-and-mechanism-project\extended-simulation-1\data\CuratedModel1_sim_results.pkl


In [37]:
# save the results
save_data(
    notebook_config=notebook_config,
    data=sim_results_df,
    data_name="sim_results_df",
    data_format="pkl",
    verbose=1,
)

# save the results
save_data(
    notebook_config=notebook_config,
    data=sim_results,
    data_name="sim_results",
    data_format="pkl",
    verbose=1,
)

Data saved at I:\My Drive\DAWSON PHD PROJECT\Biomarker Data Repository\data\data-and-mechanism-project\extended-simulation-1\data\CuratedModel1_sim_results_df.pkl
Data saved at I:\My Drive\DAWSON PHD PROJECT\Biomarker Data Repository\data\data-and-mechanism-project\extended-simulation-1\data\CuratedModel1_sim_results.pkl
